In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import cv2

In [3]:
df = pd.read_csv("ICBHI 2017/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv")
df.head()

,101,URTI
0,102,Healthy
1,103,Asthma
2,104,COPD
3,105,URTI
4,106,COPD


In [4]:
sr_no = {'101':'URTI'}
for i, j in zip(df['101'].unique(), df['URTI']):
    sr_no[str(i)] = j

In [5]:
sr_no.keys()

dict_keys(['101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226'])

In [6]:
import os
sound_files = os.listdir('Mel Spectrogram')

In [7]:
req_file_names = []

for i in sound_files:
      req_file_names.append([i])

req_file_names

[['101_1b1_Al_sc_Meditron.png'],
 ['101_1b1_Pr_sc_Meditron.png'],
 ['102_1b1_Ar_sc_Meditron.png'],
 ['103_2b2_Ar_mc_LittC2SE.png'],
 ['104_1b1_Al_sc_Litt3200.png'],
 ['104_1b1_Ar_sc_Litt3200.png'],
 ['104_1b1_Ll_sc_Litt3200.png'],
 ['104_1b1_Lr_sc_Litt3200.png'],
 ['104_1b1_Pl_sc_Litt3200.png'],
 ['104_1b1_Pr_sc_Litt3200.png'],
 ['105_1b1_Tc_sc_Meditron.png'],
 ['106_2b1_Pl_mc_LittC2SE.png'],
 ['106_2b1_Pr_mc_LittC2SE.png'],
 ['107_2b3_Al_mc_AKGC417L.png'],
 ['107_2b3_Ar_mc_AKGC417L.png'],
 ['107_2b3_Ll_mc_AKGC417L.png'],
 ['107_2b3_Lr_mc_AKGC417L.png'],
 ['107_2b3_Pl_mc_AKGC417L.png'],
 ['107_2b3_Pr_mc_AKGC417L.png'],
 ['107_2b3_Tc_mc_AKGC417L.png'],
 ['107_2b4_Al_mc_AKGC417L.png'],
 ['107_2b4_Ar_mc_AKGC417L.png'],
 ['107_2b4_Ll_mc_AKGC417L.png'],
 ['107_2b4_Lr_mc_AKGC417L.png'],
 ['107_2b4_Pl_mc_AKGC417L.png'],
 ['107_2b4_Pr_mc_AKGC417L.png'],
 ['107_2b4_Tc_mc_AKGC417L.png'],
 ['107_2b5_Al_mc_AKGC417L.png'],
 ['107_2b5_Ar_mc_AKGC417L.png'],
 ['107_2b5_Ll_mc_AKGC417L.png'],
 ['107_2b5

In [8]:
labels = []
for i in range(len(req_file_names)):
    req_file_names[i].append(sr_no[req_file_names[i][0][:3]])
    labels.append(sr_no[req_file_names[i][0][:3]])

In [9]:
x = []
for i in req_file_names:
    img = cv2.imread('Mel Spectrogram/'+i[0])
    x.append(img)

x = np.array(x)
print(x.shape)

(920, 576, 1008, 3)


In [10]:
y = np.array(labels)
y.shape

(920,)

In [11]:
unique = pd.get_dummies(y).columns
y = np.array(pd.get_dummies(y))
print(unique, y.shape)

Index(['Asthma', 'Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy', 'LRTI',
       'Pneumonia', 'URTI'],
      dtype='object') (920, 8)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape)

(736, 576, 1008, 3) (184, 576, 1008, 3) (736, 8)


In [21]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(576, 1008, 3)))
model.add(keras.layers.MaxPooling2D((3,3)))
model.add(keras.layers.Conv2D(128, (5,5), activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(64, (5,5), activation='relu'))
model.add(keras.layers.MaxPooling2D((3,3)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(8, activation='softmax'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 574, 1006, 64)     1792      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 191, 335, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 187, 331, 128)     204928    
                                                                 
 dropout_12 (Dropout)        (None, 187, 331, 128)     0         
                                                                 
 conv2d_31 (Conv2D)          (None, 183, 327, 64)      204864    
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 61, 109, 64)      0         
 g2D)                                                 

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [47]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10


TypeError: ignored